## Zaimportowanie potrzebnych bibliotek oraz plików z kodem źródłowym

In [ ]:
from dataset import Dataset
from utils import split_data
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from strategies import MarginSampling
from train import ActiveLearner

## Konfiguracja hiperparametrów

In [2]:
CONFIG = {
    "initial_size": 0.1,
    "n_queries": 100,
    "query_size": 10,
    "test_size": 0.2,
}

## Wczytanie zbioru danych i podział


In [ ]:
dataset = Dataset("wine")
X, y = dataset.get()

X_init, y_init, X_pool, y_pool, X_test, y_test = split_data(X, y, CONFIG["initial_size"], CONFIG["test_size"])

14
128


## Wybór modelu i strategii aktywnego uczenia

In [ ]:
model = MLPClassifier()
strategy = MarginSampling()

## Aktywne uczenie się za pomocą klasy ActiveLearner

In [6]:
active_learner = ActiveLearner(model, strategy)
results = active_learner.train(X_init, y_init, X_pool, y_pool, X_test, y_test, CONFIG["n_queries"], CONFIG["query_size"])
print(results)

  0%|          | 0/100 [00:00<?, ?it/s]d:\zum\.venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\zum\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
  3%|▎         | 3/100 [00:00<00:06, 14.93it/s]d:\zum\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\zum\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and th

[{'accuracy': 0.5555555555555556, 'f1_macro': 0.5122938064114534, 'precision': 0.5898692810457516, 'recall': 0.580952380952381}, {'accuracy': 0.5555555555555556, 'f1_macro': 0.4263322884012539, 'precision': 0.3682539682539683, 'recall': 0.5238095238095238}, {'accuracy': 0.8055555555555556, 'f1_macro': 0.8002645502645502, 'precision': 0.8090909090909092, 'recall': 0.8142857142857144}, {'accuracy': 0.9722222222222222, 'f1_macro': 0.9740129935032483, 'precision': 0.9777777777777779, 'recall': 0.9722222222222222}, {'accuracy': 0.9444444444444444, 'f1_macro': 0.9431841096995361, 'precision': 0.9500000000000001, 'recall': 0.9388888888888888}, {'accuracy': 0.7777777777777778, 'f1_macro': 0.7688060731538991, 'precision': 0.8141025641025642, 'recall': 0.7999999999999999}, {'accuracy': 0.4166666666666667, 'f1_macro': 0.31547619047619047, 'precision': 0.3, 'recall': 0.43333333333333335}, {'accuracy': 0.4444444444444444, 'f1_macro': 0.36738906088751294, 'precision': 0.47130647130647124, 'recall': 